## Instalaciones

In [1]:
%%capture
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth
!pip install -U bitsandbytes
!pip install Wikipedia-API

# Generación de preguntas con `Llama-3.1-8B-Instruct`

In [2]:
from unsloth import FastLanguageModel
from transformers import TextStreamer
from unsloth.chat_templates import get_chat_template


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct",
    max_seq_length = 8096,
    load_in_4bit = True
)

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
    mapping = {"role": "from", "content": "value", "user": "human", "assistant": "gpt"}
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.8: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSN

In [3]:
import wikipediaapi

# Initializate Wikipedia object
wikipedia = wikipediaapi.Wikipedia(user_agent='QuestionGeneration (francoartico0@gmail.com)', language='es')

def clean_wikipedia_article(title, sections_to_remove):
    """
    Fetches a Wikipedia article and removes specified sections.
    """
    page = wikipedia.page(title)

    if page.exists():
      for section in sections_to_remove:
          section_to_remove = page.section_by_title(section)
          if section_to_remove:
            page.sections.remove(section_to_remove)
      return page.text
    return None


In [4]:
sections_to_remove =["Enlaces externos", "Bibliografía", "Referencias", "Notas", "Véase también", "Notas y referencias", "Otras lecturas"]

article = clean_wikipedia_article(title="Bill Gates", sections_to_remove=sections_to_remove)
print(article)

William Henry Gates III (Seattle, 28 de octubre de 1955), conocido como Bill Gates, es un magnate empresarial, desarrollador de software, inversor, autor y filántropo estadounidense. Es cofundador de Microsoft, junto con su difunto amigo de la infancia Paul Allen. Durante su carrera en Microsoft, Gates ocupó los cargos de presidente, director ejecutivo (CEO), presidente y arquitecto jefe de software, además de ser el mayor accionista individual hasta mayo de 2014. Fue uno de los principales empresarios de la revolución de las microcomputadoras de las décadas de 1970 y 1980.
Su fortuna se calcula en 129 000 millones de dólares (2023) según la revista Forbes, hecho que lo colocó como el cuarto hombre más rico del mundo.[3]​ Antes del estallido de la burbuja de las punto com, su patrimonio neto ascendió a 114 100 millones de dólares, lo que lo convirtió en la décima persona más rica en toda la historia de la humanidad.[4]​
Es uno de los empresarios más conocidos que surgieron durante los 

In [5]:
messages = [
    {"from": "gpt", "value": "Eres un generador de preguntas de lectura comprensiva. Tu tarea es crear preguntas basadas en un texto dado, sin incluir respuestas. Solo debes generar preguntas incluyendo preguntas de verdadero o falso."},
    {"from": "human", "value": f"Genera exactamente 5 preguntas sobre el siguiente texto, sin incluir respuestas:\n\n{article}"},
]

inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")

print(f"Number of tokens: {len(inputs[0])}")

Number of tokens: 3041


In [6]:
text_streamer = TextStreamer(tokenizer, skip_prompt=True) # real-time output, more memory-efficient
_ = model.generate(input_ids=inputs, streamer=text_streamer, max_new_tokens=1024, use_cache=True)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Aquí te presento 5 preguntas basadas en el texto proporcionado:

1. ¿En qué año y lugar nació Bill Gates?
Respuesta: 1955, en Seattle, Washington.

2. ¿Cuál fue el nombre de la empresa fundada por Bill Gates y Paul Allen en 1975?
Respuesta: Microsoft.

3. ¿Qué fue lo que motivó a Bill Gates a dejar su cargo como director ejecutivo de Microsoft en 2008?
Respuesta: Para dedicarse por completo a su fundación.

4. ¿Qué premio recibió Bill Gates en 2016?
Respuesta: Medalla Presidencial de la Libertad.

5. ¿Cuánto se calcula la fortuna de Bill Gates en 2023 según la revista Forbes?
Respuesta: 129 000 millones de dólares.<|eot_id|>


🌐 **Fuente:** https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-Inference.ipynb